In [5]:
import time
import os
import pandas as pd
import numpy as np
import math
from numpy import linalg as LA
from datetime import datetime
from datetime import timedelta
import geopy.distance

In [31]:
class SpeedExtracter:
    """extract speed of given time interval.
    
    Parameters:
    file_name: the file name of csv file contains GPS only in target road segment.
    data_in_road_boundaries: the dataframe version of file above.
    data_in_requested_period_and_road_boundaries = dataframe contains GPS only in target road segment and given time peirod.
    
    """
    def __init__(self, file_name=""):
        self.file_name = file_name
        self.data_in_road_boundaries = self.import_csv_into_me()
        self.data_in_requested_period_and_road_boundaries = None

    def import_csv_into_me(self):
        """import csv file of given day as pandas dataframe.
        
        Returns
        ----------
        data: the imported dataframe.
        
        """
        data = pd.read_csv(self.file_name)
        return data

    def get_coordinate_array_from_dataframe_row(self, row):
        """make a numpy array of coordinates from row of dataframe.
        
        Parameters
        ----------
        row: a row of dataframe
        
        Returns
        ----------
        coordinate: the numpy array of coordinate.
        
        """
        coordinate = []
        coordinate.append(row['Longitude'])
        coordinate.append(row['Latitude'])
        coordinate = np.array(coordinate)
        return coordinate

    def is_timestamp_in_requested_period(self, timestamp, time_start, interval):
        """determine if a timestamp is in requested time period.
        
        Parameters
        ----------
        timestamp: type string, timestamp in unix format.
        time_start: the start time of requested time period, with format '%Y-%m-%d %H:%M:%S'.
        interval: the timespan of requested time period, type int, in second unit.
        
        Returns
        ----------
        type Bolean, ture if the timestamp is in requested time period, else false.
        
        """
        timestamp = int(timestamp)
        time_start_datetime = datetime.strptime(time_start, '%Y-%m-%d %H:%M:%S')
        timestamp_datetime = datetime.utcfromtimestamp(timestamp)
        difference_timedelta = timestamp_datetime - time_start_datetime
        difference_total_seconds = difference_timedelta.total_seconds()
        if (difference_total_seconds > 0) and (difference_total_seconds < interval):
            return True
        else:
            return False

    def get_df_in_requested_period(self, time_start, interval):
        """get the dataframe with all rows lacate in requested time period.
        
        Parameters
        ----------
        time_start: the start time to extract speed in one day, with format '%Y-%m-%d %H:%M:%S'.
        interval: the time interval to caculate speed, type int, in second unit.
        
        Returns
        ----------
        self.data_in_requested_period_and_road_boundaries: variable of class data_in_requested_period_and_road_boundaries.
        
        """
        self.data_in_requested_period_and_road_boundaries = self.data_in_road_boundaries
        self.data_in_requested_period_and_road_boundaries['Is_in_requested_period'] =             self.data_in_requested_period_and_road_boundaries.apply(
                lambda row: self.is_timestamp_in_requested_period(row['Timestamp'], time_start, interval), axis=1)
        self.data_in_requested_period_and_road_boundaries = self.data_in_requested_period_and_road_boundaries[
            self.data_in_requested_period_and_road_boundaries['Is_in_requested_period'] == True]
        return self.data_in_requested_period_and_road_boundaries

    def calculate_distance_based_on_coordinate(self, coords_1, coords_2):
        """calculate the distance between two coordinates.
        
        Parameters
        ----------
        coords_1: coordinate of one point, in numpy array format.
        coords_2: coordinate of another point, in numpy array format.
        
        Returns
        ----------
        distance_in_km: distance between two coordinates
        
        """
        distance_in_km = geopy.distance.vincenty(coords_1, coords_2).km
        return distance_in_km

    def get_space_mean_speed_for_target_road_segment(self, time_start, interval=300, is_from_south=True):
        """get space mean speed for target road segment at given time.

        Parameters
        ----------
        time_start: the start time to extract speed in one day, with format '%Y-%m-%d %H:%M:%S'.
        interval: the time interval to caculate speed, type int, in second unit.
        is_from_south: whether this point origins from south.

        Returns
        ----------
        space_mean_speed: the calculated space mean speed for target road segment at given time.
        num_of_drivers_considered: the number of drivers considered to calcalate the speed.
        """
        directional_df = self.get_df_in_requested_period(time_start, interval)
        directional_df = directional_df[directional_df['Is_from_south'] == is_from_south]
        distance_sum = 0
        travel_time_sum = 0
        space_mean_speed = 0
        for order in directional_df['Order_ID'].unique():
            trip_directional_df = directional_df[directional_df['Order_ID'] == order].sort_values(by=['Timestamp'])
            latitude_of_tail_point_in_one_trip = trip_directional_df.tail(1)['Latitude'].values
            longitude_of_tail_point_in_one_trip = trip_directional_df.tail(1)['Longitude'].values
            latitude_of_head_point_in_one_trip = trip_directional_df.head(1)['Latitude'].values
            longitude_of_head_point_in_one_trip = trip_directional_df.head(1)['Longitude'].values
            coords_1 = [latitude_of_tail_point_in_one_trip, longitude_of_tail_point_in_one_trip]
            coords_2 = [latitude_of_head_point_in_one_trip, longitude_of_head_point_in_one_trip]
            distance_km = self.calculate_distance_based_on_coordinate(coords_1, coords_2)
            travel_time_second = trip_directional_df.tail(1)['Timestamp'].values - trip_directional_df.head(1)[
                'Timestamp'].values
            travel_time_hour = travel_time_second / 3600.0
            distance_sum += distance_km
            travel_time_sum += travel_time_hour
        if travel_time_sum == 0:
            return None, 0
        else:
            space_mean_speed = distance_sum / travel_time_sum
        num_of_drivers_considered = len(directional_df['Order_ID'].unique())
        return space_mean_speed[0], num_of_drivers_considered

    def get_time_mean_speed_for_target_road_segment(self, time_start, interval=300, is_from_south=True):
        """get time mean speed for target road segment at given time.

        Parameters
        ----------
        time_start: the start time to extract speed in one day, with format '%Y-%m-%d %H:%M:%S'.
        interval: the time interval to caculate speed, type int, in second unit.
        is_from_south: whether this point origins from south.

        Returns
        ----------
        time_mean_speed: the calculated time mean speed for target road segment at given time.
        num_of_drivers_considered: the number of drivers considered to calcalate the speed.
        """
        directional_df = self.get_df_in_requested_period(time_start, interval)
        directional_df = directional_df[directional_df['Is_from_south'] == is_from_south]
        drivers_speed = []
        time_mean_speed = 0
        for order in directional_df['Order_ID'].unique():
            trip_directional_df = directional_df[directional_df['Order_ID'] == order].sort_values(by=['Timestamp'])
            latitude_of_tail_point_in_one_trip = trip_directional_df.tail(1)['Latitude'].values
            longitude_of_tail_point_in_one_trip = trip_directional_df.tail(1)['Longitude'].values
            latitude_of_head_point_in_one_trip = trip_directional_df.head(1)['Latitude'].values
            longitude_of_head_point_in_one_trip = trip_directional_df.head(1)['Longitude'].values
            coords_1 = [latitude_of_tail_point_in_one_trip, longitude_of_tail_point_in_one_trip]
            coords_2 = [latitude_of_head_point_in_one_trip, longitude_of_head_point_in_one_trip]
            distance_km = self.calculate_distance_based_on_coordinate(coords_1, coords_2)
            travel_time_second = trip_directional_df.tail(1)['Timestamp'].values - trip_directional_df.head(1)[
                'Timestamp'].values
            travel_time_hour = travel_time_second / 3600.0
            if travel_time_hour != 0:
                speed = distance_km / travel_time_hour
                drivers_speed.append(speed)
        if len(drivers_speed)==0:
            return None,0
        else:
            time_mean_speed = sum(drivers_speed)/len(drivers_speed)
            num_of_drivers_considered = len(drivers_speed)
            return time_mean_speed[0], num_of_drivers_considered

In [32]:
class HistoricalSpeedExtracter:
    """extract speed for all historical GPS files, this class works as a wrap-up based on SpeedExtracter.
    
    Parameters
    ----------
    time_start: the start time to extract speed in one day, with format '%Y-%m-%d %H:%M:%S'.
    interval: the time interval to caculate speed, type int, in second unit.
    historical_speed: extracted speed for all files, in list format.
    
    """
    def __init__(self, start_time='00:00:04', interval=300):
        self.start_time = start_time
        self.interval = interval
        self.historical_speed = None

    def import_csv(self, file):
        """import csv file
        
        Parameters:
        file: file path of csv to be imported.
        
        Returns:
        data: dataframe type.
        """
        columns_names = ['Driver_ID', 'Order_ID', 'Timestamp', 'Longitude', 'Latitude']
        data = pd.read_csv(file, header=None, names=columns_names)
        return data

    def get_timespan_of_one_file(self, data):
        """get the timespan of one dataframe.
        
        Parameters:
        data: dataframe with column 'Timestamp'.
        
        Returns:
        time_start: the start time of this dataframe, in datetime format.
        time_end: the end time of this dataframe, in datetime format.
        
        """
        time_start = data['Timestamp'].min()
        time_end = data['Timestamp'].max()
        time_start = datetime.utcfromtimestamp(time_start)
        time_end = datetime.utcfromtimestamp(time_end)
        return time_start, time_end

    def get_start_time_interested(self, data):
        """get the actual start time to calculate speed based on given data and start time input.
        
        Parameters:
        data: dataframe with column 'Timestamp'.
        
        Returns:
        time_base: the actual start time to calculate speed.
        """
        time_start, time_end = self.get_timespan_of_one_file(data)
        time_base = self.start_time
        time_base = time_start.strftime('%Y-%m-%d') + ' ' + time_base
        time_base = datetime.strptime(time_base, '%Y-%m-%d %H:%M:%S')
        time_base_increment = timedelta(seconds=self.interval)
        while time_base < time_start:
            time_base += time_base_increment
        time_base = time_base - time_base_increment
        return time_base

    def make_time_points_for_one_file(self, data):
        """make time points of given dataframe with interval defined by self.interval.
        
        Parameters:
        data: dataframe with column 'Timestamp'.
        
        Returns:
        time_points: a list of time point string in format '%Y-%m-%d %H:%M:%S'.
        
        """
        time_start, time_end = self.get_timespan_of_one_file(data)
        start_time_interested = self.get_start_time_interested(data)
        time_points = []
        time_points_start_datetime = start_time_interested
        time_points_end_datetime = time_end
        time_point = time_points_start_datetime
        time_increment = timedelta(seconds=self.interval)
        while time_point <= time_points_end_datetime:
            time_point_string = time_point.strftime('%Y-%m-%d %H:%M:%S')
            time_points.append(time_point_string)
            time_point += time_increment
        return time_points

    def extract_speed_for_one_file(self, file_name, is_from_south, speed_type):
        """extract speed for one GPS file.
        
        Parameters:
        file_name: file name of GPS csv file which contains GPS data points only in the target road segment.
        is_from_south: this parameter determines which direction of speed to calculate. True for Northbound speed, else Southbound.
        speed_type: string type, 'space_mean_speed' for calculating space mean speed, 'time_mean_speed' for calculating time mean speed.
        
        Returns:
        speed_data_for_one_file: a list of speed trend for given file.
        
        
        """
        speed_data_for_one_file = pd.DataFrame(columns=['time', 'speed', 'number_of_drivers_considered'])
        one_day_speed_extracter = SpeedExtracter(file_name=file_name)
        data = one_day_speed_extracter.data_in_road_boundaries
        time_points = self.make_time_points_for_one_file(data)
        for time_point in time_points:
            if speed_type == 'time_mean_speed':
                time_point_speed, number_of_drivers_considered = one_day_speed_extracter.get_time_mean_speed_for_target_road_segment(
                    time_point, interval=self.interval, is_from_south=is_from_south)
            else:
                time_point_speed, number_of_drivers_considered = one_day_speed_extracter.get_space_mean_speed_for_target_road_segment(
                    time_point, interval=self.interval, is_from_south=is_from_south)                
            time_point = datetime.strptime(time_point, '%Y-%m-%d %H:%M:%S')
            # change time zone from UTC to Beijing
            time_point = time_point + timedelta(hours=8)
            time_point = time_point.strftime('%Y-%m-%d %H:%M:%S')
            print(time_point,time_point_speed,number_of_drivers_considered)
            speed_data_for_one_time_point = pd.DataFrame([[time_point, time_point_speed, number_of_drivers_considered]],
                                                         columns=['time', 'speed', 'number_of_drivers_considered'])
            speed_data_for_one_file = speed_data_for_one_file.append(speed_data_for_one_time_point, ignore_index=True)
        return speed_data_for_one_file

    def extract_speed_for_October(self, is_from_south=True, is_all_data=False, speed_type = 'time_mean_speed'):
        """based on the method immediately above, the method extract all speed for October.
        
        Parameters:
        is_from_south: this parameter determines which direction of speed to calculate. True for Northbound speed, else Southbound.
        is_all_data: Bolean, true for extracting all speed of October, false for extracting only one day data for experiment.
        speed_type: string type, 'space_mean_speed' for calculating space mean speed, 'time_mean_speed' for calculating time mean speed.
        
        Returns:
        self.historical_speed: variable historical_speed of class, which is a list of speed of October or one day.
        """
        # put all data files in one folder
        root_path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
        if is_all_data:
            data_path = os.path.normpath(root_path + os.sep + 'data_all_target_road')
        else:
            data_path = os.path.normpath(root_path + os.sep + 'data_one_target_road')
        self.is_from_south = is_from_south
        files = os.listdir(data_path)
        speed_data_for_all_files = pd.DataFrame(columns=['time', 'speed', 'number_of_drivers_considered'])
        for file in files:
            file_path = os.path.normpath(data_path + os.sep + file)
            speed_data_for_one_file = self.extract_speed_for_one_file(file_path, is_from_south,speed_type)
            speed_data_for_all_files = speed_data_for_all_files.append(speed_data_for_one_file, ignore_index=True)
        self.historical_speed = speed_data_for_all_files
        return self.historical_speed

    def convert_extracted_data_to_csv(self):
        """convert extracted speed data to csv file.
        
        """
        output_directory = os.getcwd() + os.sep
        direction = 'to_north' if self.is_from_south else 'to_south'
        output_file_name = '{}s_interval_to_{}_historical_target_road_average_speed_data.csv'.format(
            self.interval, direction)
        self.historical_speed.to_csv(output_directory + output_file_name)
        print('the data has been written to csv!')



In [33]:
historical_extracter = HistoricalSpeedExtracter()

start_time = time.time()
historical_extracter.extract_speed_for_October(is_from_south=False,is_all_data=True,speed_type='time_mean_speed')
print("total time spend running = ",(time.time()-start_time)/3600)
historical_extracter.convert_extracted_data_to_csv()

2016-10-01 00:05:04 27.222011647567626 1
2016-10-01 00:10:04 36.21363335469366 5
2016-10-01 00:15:04 33.00376240513438 8
2016-10-01 00:20:04 26.80843831400435 23
2016-10-01 00:25:04 36.98477404583537 19
2016-10-01 00:30:04 31.03722730571148 20
2016-10-01 00:35:04 35.90889129053477 21
2016-10-01 00:40:04 33.37335508416881 19
2016-10-01 00:45:04 33.579457718432714 24
2016-10-01 00:50:04 28.355012403898602 14
2016-10-01 00:55:04 36.1716559286176 13
2016-10-01 01:00:04 31.068974844495415 21
2016-10-01 01:05:04 39.18318433965685 13
2016-10-01 01:10:04 38.54919249636069 13
2016-10-01 01:15:04 40.967116903780315 8
2016-10-01 01:20:04 37.19514022708821 7
2016-10-01 01:25:04 42.96356212930665 8
2016-10-01 01:30:04 39.082330890263485 11
2016-10-01 01:35:04 36.47625000122121 8
2016-10-01 01:40:04 39.39089913836589 5
2016-10-01 01:45:04 44.40248281428292 5
2016-10-01 01:50:04 47.32186913958547 5
2016-10-01 01:55:04 40.4800498111357 5
2016-10-01 02:00:04 39.613450795375805 7
2016-10-01 02:05:04 36.

2016-10-01 16:50:04 21.49309047041624 22
2016-10-01 16:55:04 24.064977999739874 21
2016-10-01 17:00:04 28.11053015653758 16
2016-10-01 17:05:04 25.23054508392324 14
2016-10-01 17:10:04 18.040046441234498 15
2016-10-01 17:15:04 23.760493671439146 17
2016-10-01 17:20:04 24.84915718621582 12
2016-10-01 17:25:04 25.35837461820755 16
2016-10-01 17:30:04 26.132876996369998 14
2016-10-01 17:35:04 21.562622474848272 13
2016-10-01 17:40:04 26.731621589176314 11
2016-10-01 17:45:04 26.67809019931998 19
2016-10-01 17:50:04 25.16438463264015 16
2016-10-01 17:55:04 26.28548418383594 17
2016-10-01 18:00:04 25.452044062037064 14
2016-10-01 18:05:04 24.009033199421484 22
2016-10-01 18:10:04 23.466386798794854 12
2016-10-01 18:15:04 25.841200701941453 11
2016-10-01 18:20:04 20.28926972249008 12
2016-10-01 18:25:04 20.620293068004532 16
2016-10-01 18:30:04 26.26449665777766 14
2016-10-01 18:35:04 19.712488432497313 13
2016-10-01 18:40:04 24.320912606433392 17
2016-10-01 18:45:04 23.981220206446565 17
20

2016-10-02 09:45:04 22.79206504868186 6
2016-10-02 09:50:04 25.862669672400436 9
2016-10-02 09:55:04 23.426752258614325 13
2016-10-02 10:00:04 28.34285375335166 14
2016-10-02 10:05:04 21.83558351927133 13
2016-10-02 10:10:04 32.65671437759182 11
2016-10-02 10:15:04 29.11487014889046 15
2016-10-02 10:20:04 27.351262225430105 9
2016-10-02 10:25:04 26.70518223929838 18
2016-10-02 10:30:04 29.109286717860297 17
2016-10-02 10:35:04 28.899863486569267 10
2016-10-02 10:40:04 17.23741921665352 11
2016-10-02 10:45:04 27.418272973855764 12
2016-10-02 10:50:04 22.083596570507122 12
2016-10-02 10:55:04 22.906714084981733 15
2016-10-02 11:00:04 19.659946229698065 11
2016-10-02 11:05:04 18.480634390167165 9
2016-10-02 11:10:04 25.924807569070172 21
2016-10-02 11:15:04 21.867133015613117 17
2016-10-02 11:20:04 21.15957124518687 16
2016-10-02 11:25:04 23.865322572108465 16
2016-10-02 11:30:04 30.502352322966402 21
2016-10-02 11:35:04 28.01923200945645 11
2016-10-02 11:40:04 23.942162008788333 11
2016-

2016-10-03 02:15:04 40.66165263791916 3
2016-10-03 02:20:04 45.03229561373307 2
2016-10-03 02:25:04 36.44847768447185 8
2016-10-03 02:30:04 52.07809518437746 7
2016-10-03 02:35:04 43.38886694669903 3
2016-10-03 02:40:04 37.05847583352834 8
2016-10-03 02:45:04 36.92553216714242 7
2016-10-03 02:50:04 27.319906301280298 2
2016-10-03 02:55:04 39.01736219927903 8
2016-10-03 03:00:04 46.71670882148844 5
2016-10-03 03:05:04 42.39699151507473 2
2016-10-03 03:10:04 42.468641554570546 4
2016-10-03 03:15:04 39.3635416979565 2
2016-10-03 03:20:04 51.771361568118294 3
2016-10-03 03:25:04 55.325840577199585 2
2016-10-03 03:30:04 40.561153764231506 4
2016-10-03 03:35:04 None 0
2016-10-03 03:40:04 45.93584062856446 2
2016-10-03 03:45:04 59.71647759385306 1
2016-10-03 03:50:04 51.790224990636595 1
2016-10-03 03:55:04 55.77661361878946 1
2016-10-03 04:00:04 41.82219015568868 1
2016-10-03 04:05:04 None 0
2016-10-03 04:10:04 54.27103826884991 2
2016-10-03 04:15:04 47.778676692863705 1
2016-10-03 04:20:04 

2016-10-03 19:05:04 25.557347813752223 18
2016-10-03 19:10:04 29.45395941230558 20
2016-10-03 19:15:04 24.386257446127384 23
2016-10-03 19:20:04 23.643055112147813 21
2016-10-03 19:25:04 25.721198152229732 19
2016-10-03 19:30:04 25.092512486971383 16
2016-10-03 19:35:04 27.158640828222868 21
2016-10-03 19:40:04 29.53141745353299 16
2016-10-03 19:45:04 26.673422402689987 21
2016-10-03 19:50:04 28.074161815625825 19
2016-10-03 19:55:04 26.89421637862412 19
2016-10-03 20:00:04 23.82141916457723 17
2016-10-03 20:05:04 25.631143180314776 16
2016-10-03 20:10:04 22.816752591888132 15
2016-10-03 20:15:04 21.410889491419866 15
2016-10-03 20:20:04 20.28050907771603 16
2016-10-03 20:25:04 22.204244856556286 16
2016-10-03 20:30:04 28.175807255122347 14
2016-10-03 20:35:04 30.212438877617586 10
2016-10-03 20:40:04 19.745209427187135 20
2016-10-03 20:45:04 19.43492530423489 23
2016-10-03 20:50:04 17.906092100706353 18
2016-10-03 20:55:04 27.420854039207182 15
2016-10-03 21:00:04 27.734348965903422 1

...

2016-10-27 18:00:04 18.668673726858557 11
2016-10-27 18:05:04 26.45585323732738 11
2016-10-27 18:10:04 19.81754951775635 12
2016-10-27 18:15:04 20.413630934632977 11
2016-10-27 18:20:04 17.922958345187897 14
2016-10-27 18:25:04 21.040205044142006 13
2016-10-27 18:30:04 14.418556415367874 20
2016-10-27 18:35:04 19.764016499130644 27
2016-10-27 18:40:04 22.16981256991288 10
2016-10-27 18:45:04 18.862175590486494 15
2016-10-27 18:50:04 23.2525847107284 19
2016-10-27 18:55:04 17.70773954698537 16
2016-10-27 19:00:04 22.01742626562399 16
2016-10-27 19:05:04 22.225171153865606 27
2016-10-27 19:10:04 25.617594722003712 17
2016-10-27 19:15:04 21.523504863859753 23
2016-10-27 19:20:04 20.769351663617062 21
2016-10-27 19:25:04 19.764300850642815 19
2016-10-27 19:30:04 19.10383783500192 16
2016-10-27 19:35:04 24.76825581698266 27
2016-10-27 19:40:04 25.73162733204625 28
2016-10-27 19:45:04 19.65492635658763 23
2016-10-27 19:50:04 21.991158926904546 21
2016-10-27 19:55:04 19.113094442540934 24
201

2016-10-28 11:05:04 26.06769402305001 16
2016-10-28 11:10:04 23.180061543359237 10
2016-10-28 11:15:04 19.90605194709883 13
2016-10-28 11:20:04 21.537760106910774 18
2016-10-28 11:25:04 20.154440970028126 14
2016-10-28 11:30:04 25.383187766860086 14
2016-10-28 11:35:04 24.102737237844117 16
2016-10-28 11:40:04 20.75201925513572 18
2016-10-28 11:45:04 21.69445043910999 21
2016-10-28 11:50:04 29.12993643461502 7
2016-10-28 11:55:04 21.60707108189511 19
2016-10-28 12:00:04 21.159870101298193 8
2016-10-28 12:05:04 21.3644204676148 11
2016-10-28 12:10:04 19.265709093226928 18
2016-10-28 12:15:04 23.947878601266268 22
2016-10-28 12:20:04 20.751562836355795 21
2016-10-28 12:25:04 22.744881982226783 13
2016-10-28 12:30:04 17.89646523478587 5
2016-10-28 12:35:04 19.038181500121834 11
2016-10-28 12:40:04 19.562575548728013 11
2016-10-28 12:45:04 21.569331473278517 10
2016-10-28 12:50:04 27.05510260851942 16
2016-10-28 12:55:04 20.737661174843357 14
2016-10-28 13:00:04 26.496732984168133 16
2016-

2016-10-29 03:45:04 42.573005248106234 5
2016-10-29 03:50:04 40.135929438660646 3
2016-10-29 03:55:04 43.50004830591128 4
2016-10-29 04:00:04 44.05530648197964 2
2016-10-29 04:05:04 59.32813658925868 1
2016-10-29 04:10:04 55.746290764786195 1
2016-10-29 04:15:04 None 0
2016-10-29 04:20:04 37.6838154217886 1
2016-10-29 04:25:04 26.23457324579661 3
2016-10-29 04:30:04 20.15164704693453 2
2016-10-29 04:35:04 34.89151577612758 2
2016-10-29 04:40:04 None 0
2016-10-29 04:45:04 43.88980155320863 2
2016-10-29 04:50:04 57.49452183667023 2
2016-10-29 04:55:04 49.54757500275669 2
2016-10-29 05:00:04 37.73669305932224 5
2016-10-29 05:05:04 36.35932977590163 4
2016-10-29 05:10:04 41.03864378204671 3
2016-10-29 05:15:04 40.03352617068029 1
2016-10-29 05:20:04 27.462896838171233 1
2016-10-29 05:25:04 None 0
2016-10-29 05:30:04 47.79259064736735 3
2016-10-29 05:35:04 48.08644608666191 2
2016-10-29 05:40:04 None 0
2016-10-29 05:45:04 50.380892330495946 3
2016-10-29 05:50:04 48.15871645707518 3
2016-10-

2016-10-29 20:25:04 24.470337316332873 26
2016-10-29 20:30:04 23.128733568594644 31
2016-10-29 20:35:04 26.81306250962959 16
2016-10-29 20:40:04 28.3888466369717 16
2016-10-29 20:45:04 24.238860179069526 22
2016-10-29 20:50:04 26.047043468728337 22
2016-10-29 20:55:04 17.182392632181188 17
2016-10-29 21:00:04 26.875348818489012 28
2016-10-29 21:05:04 24.053470797842007 31
2016-10-29 21:10:04 24.082488902306174 21
2016-10-29 21:15:04 23.444363982553515 13
2016-10-29 21:20:04 22.527211304180625 13
2016-10-29 21:25:04 18.613614440041623 13
2016-10-29 21:30:04 19.90627158156883 14
2016-10-29 21:35:04 14.832634990770313 21
2016-10-29 21:40:04 24.9897164062693 27
2016-10-29 21:45:04 24.92886425156185 27
2016-10-29 21:50:04 22.49373101435911 16
2016-10-29 21:55:04 23.11062500787965 21
2016-10-29 22:00:04 25.15635188161144 19
2016-10-29 22:05:04 25.482730549537553 21
2016-10-29 22:10:04 23.358655570733823 23
2016-10-29 22:15:04 33.306764336229634 10
2016-10-29 22:20:04 28.667712506109083 11
20

2016-10-30 13:30:04 19.898486131845498 20
2016-10-30 13:35:04 25.83303101069254 24
2016-10-30 13:40:04 25.325622126504054 21
2016-10-30 13:45:04 23.192633167026436 22
2016-10-30 13:50:04 19.11448102110864 16
2016-10-30 13:55:04 24.09394078769752 19
2016-10-30 14:00:04 24.89564544739571 19
2016-10-30 14:05:04 20.542339446181202 18
2016-10-30 14:10:04 26.19605640962699 23
2016-10-30 14:15:04 20.78321810880647 17
2016-10-30 14:20:04 23.498065169940237 16
2016-10-30 14:25:04 26.362610905736442 25
2016-10-30 14:30:04 28.467114459790626 19
2016-10-30 14:35:04 19.70030500367281 17
2016-10-30 14:40:04 20.978047790240606 23
2016-10-30 14:45:04 23.12732642341469 22
2016-10-30 14:50:04 24.908109719700395 16
2016-10-30 14:55:04 22.940388205822845 22
2016-10-30 15:00:04 24.176624567935193 27
2016-10-30 15:05:04 21.50433146493809 21
2016-10-30 15:10:04 28.646592664890377 23
2016-10-30 15:15:04 23.973718322139344 14
2016-10-30 15:20:04 21.723069423916876 16
2016-10-30 15:25:04 20.23531718291817 25
20

In [34]:
historical_extracter = HistoricalSpeedExtracter()

start_time = time.time()
historical_extracter.extract_speed_for_October(is_from_south=True,is_all_data=True,speed_type='time_mean_speed')
print("total time spend running = ",(time.time()-start_time)/3600)
historical_extracter.convert_extracted_data_to_csv()

2016-10-01 00:05:04 36.223587483089474 4
2016-10-01 00:10:04 37.49961070020103 11
2016-10-01 00:15:04 35.0054062634429 15
2016-10-01 00:20:04 35.006568788621756 15
2016-10-01 00:25:04 35.94383697363239 11
2016-10-01 00:30:04 44.73751024980778 11
2016-10-01 00:35:04 36.50126311148109 17
2016-10-01 00:40:04 39.57221241991967 12
2016-10-01 00:45:04 37.77265332113978 15
2016-10-01 00:50:04 36.040204932394595 13
2016-10-01 00:55:04 30.14112601692025 13
2016-10-01 01:00:04 37.72412778761933 10
2016-10-01 01:05:04 30.49206844690735 11
2016-10-01 01:10:04 38.89954191095295 6
2016-10-01 01:15:04 37.257421948815924 6
2016-10-01 01:20:04 34.4540866841139 8
2016-10-01 01:25:04 44.13813682900519 9
2016-10-01 01:30:04 38.35010285823188 8
2016-10-01 01:35:04 39.67838862524668 4
2016-10-01 01:40:04 46.888388845054045 3
2016-10-01 01:45:04 38.85603173952137 4
2016-10-01 01:50:04 42.630967954116876 6
2016-10-01 01:55:04 32.12230014251978 2
2016-10-01 02:00:04 36.88400273967222 3
2016-10-01 02:05:04 46.6

2016-10-01 16:45:04 28.90143494340153 32
2016-10-01 16:50:04 26.4847839944891 30
2016-10-01 16:55:04 26.66610314716997 27
2016-10-01 17:00:04 28.22933211973775 37
2016-10-01 17:05:04 27.85599073769174 33
2016-10-01 17:10:04 26.75222130417653 25
2016-10-01 17:15:04 24.56444402977602 38
2016-10-01 17:20:04 24.824976726745547 28
2016-10-01 17:25:04 25.001314757287144 27
2016-10-01 17:30:04 22.680013559764934 30
2016-10-01 17:35:04 23.39073479676413 29
2016-10-01 17:40:04 24.528737417401143 26
2016-10-01 17:45:04 23.12157970492223 26
2016-10-01 17:50:04 28.15079125025902 18
2016-10-01 17:55:04 23.777503211959164 22
2016-10-01 18:00:04 25.05142625358532 29
2016-10-01 18:05:04 22.271510378559338 32
2016-10-01 18:10:04 29.206691719544732 23
2016-10-01 18:15:04 27.228735689255185 26
2016-10-01 18:20:04 27.425459781703335 28
2016-10-01 18:25:04 23.45745981207362 24
2016-10-01 18:30:04 26.308662636155997 29
2016-10-01 18:35:04 30.079525639056733 29
2016-10-01 18:40:04 26.181917379545016 22
2016-

2016-10-02 09:40:04 26.91943171028691 14
2016-10-02 09:45:04 27.886043372193573 17
2016-10-02 09:50:04 30.471444869285797 20
2016-10-02 09:55:04 23.52959335147566 29
2016-10-02 10:00:04 17.409635418392668 14
2016-10-02 10:05:04 22.24929134661642 22
2016-10-02 10:10:04 28.958290752626965 14
2016-10-02 10:15:04 23.543797803939587 13
2016-10-02 10:20:04 26.003681377935123 20
2016-10-02 10:25:04 25.598878309233584 19
2016-10-02 10:30:04 24.890619435170372 17
2016-10-02 10:35:04 23.230701127522813 16
2016-10-02 10:40:04 25.897208513897166 17
2016-10-02 10:45:04 29.052317617466915 18
2016-10-02 10:50:04 28.47348432161441 28
2016-10-02 10:55:04 21.49527054281948 25
2016-10-02 11:00:04 25.587510616841506 32
2016-10-02 11:05:04 30.877384687844238 26
2016-10-02 11:10:04 26.704932108468284 21
2016-10-02 11:15:04 19.517219426010417 29
2016-10-02 11:20:04 20.926829284984034 23
2016-10-02 11:25:04 22.7724325328646 19
2016-10-02 11:30:04 24.028089496607638 23
2016-10-02 11:35:04 27.77147802347715 30


2016-10-03 02:10:04 41.960529920338246 1
2016-10-03 02:15:04 40.90502219087507 4
2016-10-03 02:20:04 None 0
2016-10-03 02:25:04 53.2960646427389 2
2016-10-03 02:30:04 49.99849904042082 4
2016-10-03 02:35:04 39.61086147499892 4
2016-10-03 02:40:04 46.26986697628977 3
2016-10-03 02:45:04 50.31190367622908 4
2016-10-03 02:50:04 29.42937213357928 4
2016-10-03 02:55:04 34.29572083970065 2
2016-10-03 03:00:04 33.0639195322956 5
2016-10-03 03:05:04 46.036118552098934 3
2016-10-03 03:10:04 59.04688067645305 1
2016-10-03 03:15:04 47.5183708809179 1
2016-10-03 03:20:04 None 0
2016-10-03 03:25:04 39.12213002614163 1
2016-10-03 03:30:04 51.611432311600275 3
2016-10-03 03:35:04 45.28027241721556 3
2016-10-03 03:40:04 0.0 1
2016-10-03 03:45:04 38.89803211959647 1
2016-10-03 03:50:04 None 0
2016-10-03 03:55:04 None 0
2016-10-03 04:00:04 59.47471234996216 1
2016-10-03 04:05:04 None 0
2016-10-03 04:10:04 60.71124491661591 2
2016-10-03 04:15:04 58.62192853810632 1
2016-10-03 04:20:04 44.22382888722124 2

2016-10-03 19:05:04 20.971651663692853 35
2016-10-03 19:10:04 23.31897277771611 30
2016-10-03 19:15:04 23.776772371593538 23
2016-10-03 19:20:04 21.350662893437086 23
2016-10-03 19:25:04 23.4221512509044 17
2016-10-03 19:30:04 19.328437318415315 19
2016-10-03 19:35:04 19.133021191576034 23
2016-10-03 19:40:04 21.681954324366675 22
2016-10-03 19:45:04 25.344691974539813 18
2016-10-03 19:50:04 23.809202281943506 27
2016-10-03 19:55:04 9.6123324317316 34
2016-10-03 20:00:04 23.20997676782394 26
2016-10-03 20:05:04 12.668665569168608 29
2016-10-03 20:10:04 18.62459680713134 19
2016-10-03 20:15:04 18.331796288929148 24
2016-10-03 20:20:04 14.397081889145786 33
2016-10-03 20:25:04 17.555855629543334 27
2016-10-03 20:30:04 22.16861427863805 23
2016-10-03 20:35:04 21.43660934653852 20
2016-10-03 20:40:04 22.560121933760758 15
2016-10-03 20:45:04 16.687360589915365 15
2016-10-03 20:50:04 23.41682330467447 22
2016-10-03 20:55:04 21.544872373878484 31
2016-10-03 21:00:04 20.914539027887372 27
201

...

2016-10-27 14:00:04 11.62739494994415 21
2016-10-27 14:05:04 11.900968214043505 35
2016-10-27 14:10:04 14.157978726403064 29
2016-10-27 14:15:04 16.223400240078856 24
2016-10-27 14:20:04 14.600403436615982 29
2016-10-27 14:25:04 14.156019298781711 31
2016-10-27 14:30:04 10.453911960281697 35
2016-10-27 14:35:04 11.909492603379368 30
2016-10-27 14:40:04 11.94571306767546 37
2016-10-27 14:45:04 14.54309687096662 27
2016-10-27 14:50:04 12.830787973322737 31
2016-10-27 14:55:04 14.985890889271756 38
2016-10-27 15:00:04 17.025601860271003 37
2016-10-27 15:05:04 20.060686398219 40
2016-10-27 15:10:04 17.8908788483312 39
2016-10-27 15:15:04 16.546062300363506 43
2016-10-27 15:20:04 15.822832983832036 29
2016-10-27 15:25:04 11.170094271381306 29
2016-10-27 15:30:04 14.083963053671562 27
2016-10-27 15:35:04 15.818560807783728 33
2016-10-27 15:40:04 17.89137150699176 27
2016-10-27 15:45:04 17.14913332138418 22
2016-10-27 15:50:04 17.894019257195183 26
2016-10-27 15:55:04 19.569235457612795 19
20

2016-10-28 07:05:04 33.60952523133958 3
2016-10-28 07:10:04 24.997057673193353 5
2016-10-28 07:15:04 23.518326091796975 7
2016-10-28 07:20:04 27.6950217173516 7
2016-10-28 07:25:04 24.79225654219035 10
2016-10-28 07:30:04 16.4841431295135 11
2016-10-28 07:35:04 16.13657427497486 12
2016-10-28 07:40:04 16.995764246077428 9
2016-10-28 07:45:04 15.85195136180231 7
2016-10-28 07:50:04 22.61531884073317 9
2016-10-28 07:55:04 18.691659833887336 10
2016-10-28 08:00:04 17.579822347242295 11
2016-10-28 08:05:04 9.130261245516602 9
2016-10-28 08:10:04 15.173420837156113 12
2016-10-28 08:15:04 12.623873992465088 16
2016-10-28 08:20:04 14.23828237247351 27
2016-10-28 08:25:04 14.46016221083033 20
2016-10-28 08:30:04 18.72141842357017 15
2016-10-28 08:35:04 13.224256344513293 10
2016-10-28 08:40:04 20.0623622394608 20
2016-10-28 08:45:04 25.117548505913316 21
2016-10-28 08:50:04 23.720698207558616 26
2016-10-28 08:55:04 17.406343822162462 21
2016-10-28 09:00:04 29.638014877041496 19
2016-10-28 09:0

2016-10-28 23:35:04 37.72988635377688 16
2016-10-28 23:40:04 33.436812821766054 18
2016-10-28 23:45:04 28.927464300137586 14
2016-10-28 23:50:04 35.443793195654976 13
2016-10-28 23:55:04 35.77571805136038 10
2016-10-29 00:00:04 None 0
2016-10-29 00:05:04 42.38830129557028 3
2016-10-29 00:10:04 30.673864516171797 5
2016-10-29 00:15:04 36.679182873419975 4
2016-10-29 00:20:04 36.41862316907471 19
2016-10-29 00:25:04 31.295727758409747 15
2016-10-29 00:30:04 33.582947524086634 8
2016-10-29 00:35:04 41.476421768998904 9
2016-10-29 00:40:04 33.99431016677141 11
2016-10-29 00:45:04 34.65629259359255 10
2016-10-29 00:50:04 41.94190459945371 6
2016-10-29 00:55:04 36.195501139281454 1
2016-10-29 01:00:04 34.333575008486044 7
2016-10-29 01:05:04 38.37663134403722 12
2016-10-29 01:10:04 34.24972408226213 12
2016-10-29 01:15:04 42.28844875209199 4
2016-10-29 01:20:04 43.408297869787724 2
2016-10-29 01:25:04 40.96244322271617 3
2016-10-29 01:30:04 36.55843939334443 7
2016-10-29 01:35:04 32.81134850

2016-10-29 16:25:04 19.528605166883587 35
2016-10-29 16:30:04 27.3879626692872 32
2016-10-29 16:35:04 26.70652040860431 31
2016-10-29 16:40:04 26.552167976140385 32
2016-10-29 16:45:04 30.572263144028298 26
2016-10-29 16:50:04 22.39520628362686 29
2016-10-29 16:55:04 21.19940758532041 36
2016-10-29 17:00:04 19.702209447499023 35
2016-10-29 17:05:04 22.161786523118977 31
2016-10-29 17:10:04 28.19580851986823 35
2016-10-29 17:15:04 25.59458978488966 28
2016-10-29 17:20:04 24.48161021799528 26
2016-10-29 17:25:04 25.17781837272344 27
2016-10-29 17:30:04 23.28690085781271 31
2016-10-29 17:35:04 26.18958651838718 30
2016-10-29 17:40:04 25.431543336950234 23
2016-10-29 17:45:04 21.29640940660285 35
2016-10-29 17:50:04 23.175674345667858 34
2016-10-29 17:55:04 20.13037951258647 33
2016-10-29 18:00:04 27.96188725378763 21
2016-10-29 18:05:04 23.20511448163017 31
2016-10-29 18:10:04 25.222376810394703 22
2016-10-29 18:15:04 22.134797748003834 24
2016-10-29 18:20:04 23.026562133640535 24
2016-10

2016-10-30 09:20:04 33.85410790251866 16
2016-10-30 09:25:04 23.379755569729404 14
2016-10-30 09:30:04 20.20168824773387 11
2016-10-30 09:35:04 25.98326680359837 11
2016-10-30 09:40:04 23.049432408983797 21
2016-10-30 09:45:04 26.074607233788207 23
2016-10-30 09:50:04 25.775918495365058 14
2016-10-30 09:55:04 27.113931116603364 13
2016-10-30 10:00:04 20.14075144159677 16
2016-10-30 10:05:04 25.75972400425943 9
2016-10-30 10:10:04 28.867156030725923 15
2016-10-30 10:15:04 29.80173155904831 12
2016-10-30 10:20:04 25.989882727329007 19
2016-10-30 10:25:04 24.524198275623725 13
2016-10-30 10:30:04 22.906134161618642 16
2016-10-30 10:35:04 27.21237652178589 16
2016-10-30 10:40:04 23.951675897708437 22
2016-10-30 10:45:04 28.104722981624498 20
2016-10-30 10:50:04 25.60438554647096 20
2016-10-30 10:55:04 24.457622664016284 26
2016-10-30 11:00:04 24.834062030148264 18
2016-10-30 11:05:04 22.1714000876422 19
2016-10-30 11:10:04 22.66725391796763 26
2016-10-30 11:15:04 26.1897107284674 22
2016-1